In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc
import os, warnings 
warnings.filterwarnings('ignore') 

import stan

In [2]:
path = Path('outputs_cv')
if not os.path.exists(path):
    os.makedirs(path)

In [3]:
import numpy as np
import pandas as pd
import scanpy as sc
import time
from scipy.stats import spearmanr, pearsonr


def assign_folds(adata, n_folds=5, train_percent=None, random_seed=0):
    np.random.seed(random_seed)
    if train_percent is not None:
        adata.var["fold"]=np.random.uniform(0,1, adata.n_vars)<train_percent
    else:
        adata.var["fold"]=np.random.randint(0,n_folds, adata.n_vars)
        

class ModelBase:
    def __init__(self, adata, kernel_name='kernel', layer="dca", gene_tf=None, intercept=False):
        self.adata = adata
        self.intercept = intercept

        # Added a try to deal with the gene_pw situation
        try:
            gene_tf = adata.varm['gene_tf']
        except:
            gene_tf = adata.varm['gene_pw']

        self.D = gene_tf
        self.Y = adata.to_df(layer).T
        self.genes = np.intersect1d(self.Y.index, self.D.index)
        self.D = self.D.loc[self.genes]
        self.Y = self.Y.loc[self.genes]

        self.n_folds = np.max(adata.var['fold']) + 1
        self.n_genes = len(self.genes)
        self.n_spots = adata.n_obs
        self.n_tfs = self.D.shape[1]

        self.training_genes = [adata.var.query("fold != @i").index for i in range(self.n_folds)]
        self.testing_genes = [adata.var.query("fold == @i").index for i in range(self.n_folds)]
        self.D_train = [self.D.loc[self.training_genes[i]].to_numpy() for i in range(self.n_folds)]
        self.D_test = [self.D.loc[self.testing_genes[i]].to_numpy() for i in range(self.n_folds)]
        self.Y_train = [self.Y.loc[self.training_genes[i]].to_numpy() for i in range(self.n_folds)]
        self.Y_test = [self.Y.loc[self.testing_genes[i]].to_numpy() for i in range(self.n_folds)]
        self.D = self.D.to_numpy()

        self.K = adata.obsm.get(kernel_name)
        self.svdD = []
        for i in range(self.n_folds):
            self.svdD.append(np.linalg.svd(self.D_train[i], full_matrices=False))
        self.svdK = None

        self.W = [None] * self.n_folds
        self.y_pred = [None] * self.n_folds


    def evaluate(self, fold=0, gene_set="testing"):
        if fold == -1:
            y_pred = np.asarray(self.y_pred_concat)
            Y = np.asarray(self.Y)
        else:
            W = self.W[fold]
            if gene_set == "testing":
                Y = self.Y_test[fold]
                D = self.D_test[fold]
            elif gene_set == "training":
                Y = self.Y_train[fold]
                D = self.D_train[fold]
            y_pred = D.dot(W)

        y_pred = y_pred.astype(float)
        Y = Y.astype(float)

        cor = [pearsonr(y_pred[:, spot], Y[:, spot])[0] for spot in range(self.n_spots)]
        return cor


    def fit(self, params=dict(), verbose=False, axis=0):
        t1 = time.time()
        self.params = params
        for fold in range(self.n_folds):
            self.train(fold=fold, **params)
            self.y_pred[fold] = self.D_test[fold].dot(self.W[fold])

        self.W_concat = np.mean(self.W, axis=0)
        self.y_pred_concat = pd.DataFrame(data=None, index=self.Y.index, columns=self.Y.columns)

        for fold in range(self.n_folds):
            self.y_pred_concat.loc[self.testing_genes[fold]] = self.y_pred[fold]
        t2 = time.time()
        print('Time elapsed: %.2f seconds'%(t2-t1))

        
class Stan(ModelBase):
    def __init__(self, args, **kwargs):
        super().__init__(args, **kwargs)

    def train(self, fold=0, lam2=1, lam1=5):
        if self.svdD[fold] is None:
            self.svdD[fold] = np.linalg.svd(self.D_train[fold], full_matrices=False)
        if self.svdK is None:
            uk,sk,vk = np.linalg.svd(self.K, full_matrices=True)
            sk = np.concatenate((sk, [0]*(uk.shape[0]-len(sk))))
            self.svdK = (uk,sk,vk)

        [ud,sd,vd] = self.svdD[fold]
        [uk,sk,vk] = self.svdK
        D = self.D_train[fold]
        Y = self.Y_train[fold]

        scale = np.divide(1, sd.reshape((-1,1))**2 + lam1*lam2/(lam1*sk**2+lam2).reshape(1,-1))
        W = vd.T@(np.multiply(scale, vd@D.T@Y@uk))@uk.T
        self.W[fold] = W

class Ridge(ModelBase):
    def __init__(self, args, **kwargs):
        super().__init__(args, **kwargs)

    def train(self, fold=0, lam=1):
        if self.svdD[fold] is None:
            self.svdD[fold] = np.linalg.svd(self.D_train[fold], full_matrices=False)
        (ua, sa, va) = self.svdD[fold]
        self.W[fold] = va.T.dot(np.diag(1/(lam*self.n_genes+sa**2)).dot(va)).dot(self.D_train[fold].T.dot(self.Y_train[fold]))

In [4]:
def stan_wrap(adata):
    adata = stan.add_gene_tf_matrix(adata, min_cells_proportion = 0.2, min_tfs_per_gene= 5, min_genes_per_tf= 10,
                                    gene_tf_source="hTFtarget", tf_list="humantfs", source_dir="resources/")
    stan.pixel_intensity(adata, windowsize=25)
    stan.make_kernel_from_pixel(adata, n=250, im_feats_weight=0.1)

    sc.pp.normalize_total(adata)
    adata.layers['scaled'] = np.sqrt(adata.to_df())
    stan.assign_folds(adata, n_folds=10, random_seed=0)
    
    exponents = [3, 2, 1, 0, -1, -2, -3, -4]
    
    for i in exponents:
        for j in exponents:
            print(f'STAN parameters: {10**i}, {10**j}')
            stan_model = Stan(adata, layer='scaled')
            stan_model.fit(params={'lam1':10**i, 'lam2':10**j})
            print(stan_model.params)

            cor = stan_model.evaluate(fold=-1)
            adata.obs[f'pred_cor_stan_{i}_{j}'] = cor
            print("Spot-wise correlation:" + str(round(np.nanmedian(cor), 4)))

            adata.obsm[f'tfa_stan_{i}_{j}'] = pd.DataFrame(
                stan_model.W_concat.T, index=adata.obs_names, columns=adata.uns['tf_names'])
        
        print(f'Ridge parameters: {10**i}')
        ridge_model = Ridge(adata, layer='scaled')
        ridge_model.fit(params={'lam':10**i})
        print(ridge_model.params)

        cor = ridge_model.evaluate(fold=-1)
        adata.obs[f'pred_cor_ridge_{i}'] = cor
        print("Spot-wise correlation:" + str(round(np.nanmedian(cor), 4)))

        adata.obsm[f'tfa_ridge_{i}'] = pd.DataFrame(
            ridge_model.W_concat.T, index=adata.obs_names, columns=adata.uns['tf_names'])
    return adata



### Breast

In [5]:
sample_list = ["1160920F", "CID44971", "CID4535"]
for sample_id in sample_list:
    adata = stan.read_breast_wu("data/Breast_Wu/{}.h5ad".format(sample_id))
    adata = stan_wrap(adata)
    adata.write(path / ('adata_Breast_Wu_'+sample_id+'.h5ad'))

STAN parameters: 1000, 1000
Time elapsed: 14.99 seconds
{'lam1': 1000, 'lam2': 1000}
Spot-wise correlation:0.2232
STAN parameters: 1000, 100
Time elapsed: 13.51 seconds
{'lam1': 1000, 'lam2': 100}
Spot-wise correlation:0.2232
STAN parameters: 1000, 10
Time elapsed: 13.61 seconds
{'lam1': 1000, 'lam2': 10}
Spot-wise correlation:0.223
STAN parameters: 1000, 1
Time elapsed: 14.51 seconds
{'lam1': 1000, 'lam2': 1}
Spot-wise correlation:0.2227
STAN parameters: 1000, 0.1
Time elapsed: 17.07 seconds
{'lam1': 1000, 'lam2': 0.1}
Spot-wise correlation:0.2224
STAN parameters: 1000, 0.01
Time elapsed: 15.35 seconds
{'lam1': 1000, 'lam2': 0.01}
Spot-wise correlation:0.2224
STAN parameters: 1000, 0.001
Time elapsed: 16.45 seconds
{'lam1': 1000, 'lam2': 0.001}
Spot-wise correlation:0.2224
STAN parameters: 1000, 0.0001
Time elapsed: 16.08 seconds
{'lam1': 1000, 'lam2': 0.0001}
Spot-wise correlation:0.2224
Ridge parameters: 1000
Time elapsed: 8.14 seconds
{'lam': 1000}
Spot-wise correlation:0.0328
STAN

Spot-wise correlation:0.2835
STAN parameters: 1000, 1
Time elapsed: 1.78 seconds
{'lam1': 1000, 'lam2': 1}
Spot-wise correlation:0.2836
STAN parameters: 1000, 0.1
Time elapsed: 1.74 seconds
{'lam1': 1000, 'lam2': 0.1}
Spot-wise correlation:0.2835
STAN parameters: 1000, 0.01
Time elapsed: 1.72 seconds
{'lam1': 1000, 'lam2': 0.01}
Spot-wise correlation:0.2833
STAN parameters: 1000, 0.001
Time elapsed: 1.79 seconds
{'lam1': 1000, 'lam2': 0.001}
Spot-wise correlation:0.2833
STAN parameters: 1000, 0.0001
Time elapsed: 1.75 seconds
{'lam1': 1000, 'lam2': 0.0001}
Spot-wise correlation:0.2833
Ridge parameters: 1000
Time elapsed: 1.28 seconds
{'lam': 1000}
Spot-wise correlation:0.0388
STAN parameters: 100, 1000
Time elapsed: 1.77 seconds
{'lam1': 100, 'lam2': 1000}
Spot-wise correlation:0.2792
STAN parameters: 100, 100
Time elapsed: 1.86 seconds
{'lam1': 100, 'lam2': 100}
Spot-wise correlation:0.2793
STAN parameters: 100, 10
Time elapsed: 1.83 seconds
{'lam1': 100, 'lam2': 10}
Spot-wise correla

Time elapsed: 1.68 seconds
{'lam1': 1000, 'lam2': 0.001}
Spot-wise correlation:0.2932
STAN parameters: 1000, 0.0001
Time elapsed: 1.69 seconds
{'lam1': 1000, 'lam2': 0.0001}
Spot-wise correlation:0.2933
Ridge parameters: 1000
Time elapsed: 1.19 seconds
{'lam': 1000}
Spot-wise correlation:0.0776
STAN parameters: 100, 1000
Time elapsed: 1.72 seconds
{'lam1': 100, 'lam2': 1000}
Spot-wise correlation:0.2899
STAN parameters: 100, 100
Time elapsed: 1.66 seconds
{'lam1': 100, 'lam2': 100}
Spot-wise correlation:0.2898
STAN parameters: 100, 10
Time elapsed: 1.69 seconds
{'lam1': 100, 'lam2': 10}
Spot-wise correlation:0.2895
STAN parameters: 100, 1
Time elapsed: 1.66 seconds
{'lam1': 100, 'lam2': 1}
Spot-wise correlation:0.2897
STAN parameters: 100, 0.1
Time elapsed: 1.67 seconds
{'lam1': 100, 'lam2': 0.1}
Spot-wise correlation:0.2894
STAN parameters: 100, 0.01
Time elapsed: 1.69 seconds
{'lam1': 100, 'lam2': 0.01}
Spot-wise correlation:0.2893
STAN parameters: 100, 0.001
Time elapsed: 1.97 secon

### Lymphnode

In [6]:
sample_id = "V1_Human_Lymph_Node"
adata = stan.read_visium_sge(sample_id=sample_id, min_cells=5, min_counts=5000)
adata = stan_wrap(adata)
adata.write(path / ('adata_'+sample_id+'.h5ad'))

STAN parameters: 1000, 1000
Time elapsed: 12.39 seconds
{'lam1': 1000, 'lam2': 1000}
Spot-wise correlation:0.2277
STAN parameters: 1000, 100
Time elapsed: 11.89 seconds
{'lam1': 1000, 'lam2': 100}
Spot-wise correlation:0.2277
STAN parameters: 1000, 10
Time elapsed: 12.18 seconds
{'lam1': 1000, 'lam2': 10}
Spot-wise correlation:0.2277
STAN parameters: 1000, 1
Time elapsed: 12.97 seconds
{'lam1': 1000, 'lam2': 1}
Spot-wise correlation:0.2277
STAN parameters: 1000, 0.1
Time elapsed: 12.54 seconds
{'lam1': 1000, 'lam2': 0.1}
Spot-wise correlation:0.2277
STAN parameters: 1000, 0.01
Time elapsed: 11.88 seconds
{'lam1': 1000, 'lam2': 0.01}
Spot-wise correlation:0.2276
STAN parameters: 1000, 0.001
Time elapsed: 11.98 seconds
{'lam1': 1000, 'lam2': 0.001}
Spot-wise correlation:0.2276
STAN parameters: 1000, 0.0001
Time elapsed: 11.31 seconds
{'lam1': 1000, 'lam2': 0.0001}
Spot-wise correlation:0.2276
Ridge parameters: 1000
Time elapsed: 7.96 seconds
{'lam': 1000}
Spot-wise correlation:0.1177
STA